In [72]:
import torch
import nltk
import evaluate
import multiprocessing
nltk.download('punkt')
from torch.utils.data import DataLoader, Dataset
import numpy as np
from tqdm import tqdm
import pandas as pd
from nltk import sent_tokenize
import math, re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize, word_tokenize
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datasets import load_dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments, pipeline
import argparse
import pickle
from rouge_score import rouge_scorer
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to /home/haskari/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [73]:
data = load_dataset("cnn_dailymail", '3.0.0')
article_key = 'article'
summary_key = 'highlights'

In [84]:
data = load_dataset("xsum")
article_key = 'document'
summary_key = 'summary'

In [92]:
data = load_dataset('reddit_tifu', 'long')
article_key = 'documents'
summary_key = 'tldr'
    # 80% train, 20% test + validation
train_testvalid = data['train'].train_test_split(test_size=0.2, seed=42)
# Split the 20% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)
# gather everyone if you want to have a single DatasetDict
data = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

In [101]:
data = load_dataset("argilla/news-summary")
article_key = 'text'
summary_key = 'prediction'
data = DatasetDict({
    'train': data['test'],
    'test': data['train']})

In [102]:
data=data['test']

In [103]:
data[summary_key][30]

[{'score': 1.0,
  'text': 'Highlights: The Trump presidency on March 31 at 6:19 p.m. EDT'}]

In [104]:
def tokenize(example):
    example["original_article"] = example[article_key]
    example["article"] = nltk.sent_tokenize(example[article_key])
    
    # # if args.dataset == 'news':
    # check=example[summary_key][0]['text']
    # example["highlights"] = nltk.sent_tokenize(check)
    # else:
    # example["highlights"] = nltk.sent_tokenize(example[summary_key])
        
    return example

data = data.map(tokenize, num_proc=multiprocessing.cpu_count())

In [105]:
data

Dataset({
    features: ['text', 'prediction', 'prediction_agent', 'annotation', 'annotation_agent', 'id', 'metadata', 'status', 'event_timestamp', 'metrics', 'original_article', 'article'],
    num_rows: 1000
})

In [106]:
sent_count=0
overall_count=0

for item in tqdm(data['highlights']):
    sent_count+=len(item)
    overall_count+=1
    

KeyError: "Column highlights not in the dataset. Current columns in the dataset: ['text', 'prediction', 'prediction_agent', 'annotation', 'annotation_agent', 'id', 'metadata', 'status', 'event_timestamp', 'metrics', 'original_article', 'article']"

In [107]:
sent_count=0
overall_count=0
min=float('inf')
art=0
for item in tqdm(data['article']):
    #print(item)
    sent_count=len(item)
    if sent_count==0:
        continue
    if sent_count<min:
        min=sent_count
        art=item

print(min)

100%|██████████| 1000/1000 [00:00<00:00, 2340571.43it/s]

1


In [108]:
min

1

In [109]:
art

['WASHINGTON (Reuters) - Democratic Party communications director Luis Miranda is expected to leave the Democratic National Committee on Tuesday amid fallout from the party’s email hacking scandal, a source familiar with the matter told Reuters.']

In [ ]:
sent_count/overall_count

In [ ]:
max_len=0
counts=[]
for item in dataset[summary_key]:
    # item=item[0]['text']
    splits=item.split('\n')
    counts.append(len(splits))
    if len(splits)>max_len:
        max_len=max(len(splits),max_len)
        print(splits)
print(max_len)
print(sum(counts)/len(counts))

In [ ]:
len(dataset[summary_key])

In [ ]:
%pip install -qU \
    transformers==4.31.0 \
    accelerate==0.21.0 \
    einops==0.6.1 \
    langchain==0.0.240 \
    xformers==0.0.20 \
    bitsandbytes==0.41.0